In [1]:
# -- IMPORTATION OF THE PACKAGES --
import DataFrames 
import XLSX
include("structs.jl")
include("read_data.jl")

get_network_data

In [2]:
# -- FETCHING THE PATH OF THE ROOT DIRECTORY --

root_dir = normpath(joinpath(@__FILE__,"..","..",".."))

"/Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/"

In [3]:
# -- LOADING THE EXCEL FILE CONTAINING THE NETWORK TOPOLOGY --

network_data_dir = joinpath(root_dir, "NetworkModels")
XLSX_NETWORK_PATH = joinpath(network_data_dir, "network_Nahman_Peric_2S23H.xlsx")

"/Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/NetworkModels/network_Nahman_Peric_2S23H.xlsx"

In [4]:
# -- LOADING THE EXCEL FILES CONTAINING THE DATA FOR LOAD AND PV PROFILES --

profiles_data_dir = joinpath(root_dir, "ManchesterData", "LCT_profiles")
XLSX_SUMMER_LOAD_PATH = joinpath(profiles_data_dir, "Summer_Load_Profiles.xlsx")
XLSX_WINTER_LOAD_PATH = joinpath(profiles_data_dir, "Winter_Load_Profiles.xlsx")
XLSX_PV_PATH = joinpath(profiles_data_dir, "Summer_PV_Profiles.xlsx")
XLSX_EV_PATH = joinpath(profiles_data_dir, "Winter_EV_Profiles.xlsx")
XLSX_HP_PATH = joinpath(profiles_data_dir, "Winter_EHP_Profiles.xlsx")

"/Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/ManchesterData/LCT_profiles/Winter_EHP_Profiles.xlsx"

In [19]:
# PROFILES is a list that contains the path to the significative days profiles 

function create_base_load_profiles( PROFILES::Vector{String},
                                    nb_profiles::Int64, 
                                    peak_power::Float64,
                                    peak_distribution::Vector{Float64};
                                    winter::Bool=true,
                                    seed=nothing
                                )

    # Fixing the seed to choose randomly the load profiles selected 
    if !isnothing(seed)
        Random.seed!(seed)
        id_loads = Random.rand(1:ncols, nb_profiles)
    else 
        id_loads = 1:nb_profiles
    end

    nrows, ncols = size(XLSX.readxlsx(PROFILES[1])[1][:])
    base_load_profiles = Matrix{Float64}(undef, 0, ncols)

    for file in PROFILES 
        profiles = convert(Matrix{Float64}, XLSX.readxlsx(file)[1][:])
        selected_profiles = reshape(profiles[:, id_loads], (nrows, length(id_loads)))
        base_load_profiles = [base_load_profiles; selected_profiles]
    end

    sum_user_profiles = vec(sum(base_load_profiles, dims=2))
    peak_value, index_peak = findmax(sum_user_profiles)
    initial_distribution = base_load_profiles[index_peak, :] ./
    argmax(sum(base_load_profiles, dims=2))
    # Computation to have the right distribution at peak time
    
    base_load_profiles[:, u] = base_load_profiles[:, u]/peak_value(distribution[u] * scaling_factor) * peak_power/scaling_factor
    

end


create_base_load_profiles (generic function with 1 method)

In [ ]:
function simulation_load_profiles( PROFILE_DIR::String,
    nb_profiles::Int64, 
    peak_power::Float64,
    peak_distribution::Vector{Float64};
    winter::Bool=true, 
    EV::Bool=false, 
    EHP::Bool=false,
    seed=nothing
)


end

In [5]:
# -- Verification of the above code --
network_data, network_typology = get_network_data(XLSX_NETWORK_PATH)

(DistributionNetwork(Line[Line(Edge(1, Node(1, (x = 8.525, y = 6.055)), Node(10, (x = 8.325, y = 6.026))), 0.20209156340629467, false, nothing, nothing, nothing, nothing, nothing), Line(Edge(2, Node(10, (x = 8.325, y = 6.026)), Node(20, (x = 8.825, y = 6.512))), 0.697277563098082, false, nothing, nothing, nothing, nothing, nothing), Line(Edge(3, Node(10, (x = 8.325, y = 6.026)), Node(14, (x = 7.925, y = 5.869))), 0.42970804041814203, false, nothing, nothing, nothing, nothing, nothing), Line(Edge(4, Node(10, (x = 8.325, y = 6.026)), Node(19, (x = 8.868, y = 5.783))), 0.5948932677380043, false, nothing, nothing, nothing, nothing, nothing), Line(Edge(5, Node(19, (x = 8.868, y = 5.783)), Node(20, (x = 8.825, y = 6.512))), 0.7302670744323606, false, nothing, nothing, nothing, nothing, nothing), Line(Edge(6, Node(19, (x = 8.868, y = 5.783)), Node(21, (x = 9.039, y = 5.255))), 0.5550000000000003, false, nothing, nothing, nothing, nothing, nothing), Line(Edge(7, Node(19, (x = 8.868, y = 5.783)

In [ ]:
# -- Function that prints the initial network structure -- 

function Base.show(io::IO, data::User) 
    println(io, "User bus : ")
    println(io, "--------- ")
    println(io, " - At node $(data.node.id)")
    println(io, " - V in [$(data.V_limits.V_min), $(data.V_limits.V_max)]")
    if !isnothing(data.load_profile)
        println(io, " - Total S consumption = $(sum(data.load_profile.profile .* data.load_profile.granularity))")
        println(io, " - cos(phi) = $(data.load_profile.cos_phi)")
    end
    if !isnothing(data.V_magn)
        println(io, " - V = $(data.V_magn) [pu]")
    end
    if !isnothing(data.PV_installation)
        println(io, " - PV installation : yes")
    else
        println(io, " - PV installation : no")
    end
  
    return 
end


In [ ]:
# -- DEFINITIONS OF THE PARAMETERS FOR LOAD PROFILES --

WINTER = true               # extend the summer profile
ELECTRIC_VEHICLES = false   # added to summer & winter profiles
HEAT_PUMPS = false          # added to winter profile only
PEAK_POWER = 7              # [MVA] maximum power of all the apparent demands
DELTA_T = 15                # [min] Duration of a time step in time series

In [ ]:
# -- DEFINITIONS OF THE PARAMETERS REQUIRED FOR PV --
MAX_PV_CAPACITY_PER_NODE = 0.4 # MVA 
PV_SCALE_SUMMER_WINTER = 0.1 # winter PV production wrt summer production
PV_MAX_Q = 0.3 # max Q consumed/produced wrt P_peak
PV_SLOPE = -1 # linear slope of the upper bound from the PQ diagram of PV
COS_PHI = 0.95 # P_CONSUMPTION = COS_PHI * S_CONSUMPTION


In [ ]:
for i in network_data.load_buses
    show(i)
end
